<a href="https://colab.research.google.com/github/olegkleiman/tf2/blob/master/classification_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras classification for MNIST 

In [5]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import os

In [6]:
try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  print('Running on TPU ', resolver.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(resolver)

tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

tpu_strategy = tf.distribute.TPUStrategy(resolver)  

Running on TPU  ['10.16.6.66:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.16.6.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.16.6.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


#Load the dataset

In [7]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split,
                            with_info=True,
                            as_supervised=True,
                            try_gcs=True)
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

# Create the model

In [8]:
def create_model():
  return keras.Sequential(
      [
       keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       keras.layers.Conv2D(256, 3, activation='relu'),
       keras.layers.Flatten(),
       keras.layers.Dense(256, activation='relu'),
       keras.layers.Dense(128, activation='relu'),
       keras.layers.Dense(10)
      ]
  )

# Train the model

In [ ]:
with tpu_strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                experimental_steps_per_execution=50,
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

  batch_size = 200
  steps_per_epoch = 60000
  validation_steps = 10000

  train_dataset = get_dataset(batch_size, is_training=True)
  test_dataset = get_dataset(batch_size, is_training=False)

  model.fit(train_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=5,
            validation_data=test_dataset,
            validation_steps=validation_steps)

Epoch 1/5
  300/60000 [..............................] - ETA: 39:06 - loss: 0.2164 - sparse_categorical_accuracy: 0.9301WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 300000 batches). You may need to use the repeat() function when building your dataset.
